In [13]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
df_kai_raw = pd.read_csv('kaiterra_fieldeggid_1H_20180501_20201101.csv', index_col=[0,1], parse_dates=True)['pm25']
df_gov_raw = pd.read_csv('govdata_1H_20180501_20201101.csv', index_col=[0,1], parse_dates=True)['pm25']
df_raw = pd.concat([df_gov_raw, df_kai_raw], axis=0)
df_raw

monitor_id       timestamp_round          
AnandVihar_DPCC  2018-05-01 00:00:00+05:30    68.0
                 2018-05-01 01:00:00+05:30    53.0
                 2018-05-01 02:00:00+05:30    53.0
                 2018-05-01 03:00:00+05:30    46.0
                 2018-05-01 04:00:00+05:30    48.0
                                              ... 
EAC8             2020-10-31 19:00:00+05:30     NaN
                 2020-10-31 20:00:00+05:30     NaN
                 2020-10-31 21:00:00+05:30     NaN
                 2020-10-31 22:00:00+05:30     NaN
                 2020-10-31 23:00:00+05:30     NaN
Name: pm25, Length: 1317600, dtype: float64

In [5]:
df_raw = df_raw.tz_localize(None, level=1)
df_raw

monitor_id       timestamp_round    
AnandVihar_DPCC  2018-05-01 00:00:00    68.0
                 2018-05-01 01:00:00    53.0
                 2018-05-01 02:00:00    53.0
                 2018-05-01 03:00:00    46.0
                 2018-05-01 04:00:00    48.0
                                        ... 
EAC8             2020-10-31 19:00:00     NaN
                 2020-10-31 20:00:00     NaN
                 2020-10-31 21:00:00     NaN
                 2020-10-31 22:00:00     NaN
                 2020-10-31 23:00:00     NaN
Name: pm25, Length: 1317600, dtype: float64

In [8]:
df_raw.sort_index(inplace=True)

In [9]:
start_dt = pd.Timestamp('2018-05-01 00:00:00')
end_dt = pd.Timestamp('2019-09-30 23:00:00')
df_raw = df_raw.loc[(slice(None), slice(start_dt, end_dt))]
df_raw

monitor_id     timestamp_round    
113E           2018-05-01 00:00:00      NaN
               2018-05-01 01:00:00      NaN
               2018-05-01 02:00:00      NaN
               2018-05-01 03:00:00      NaN
               2018-05-01 04:00:00      NaN
                                      ...  
Wazirpur_DPCC  2019-09-30 19:00:00    59.50
               2019-09-30 20:00:00    67.75
               2019-09-30 21:00:00    66.25
               2019-09-30 22:00:00    53.00
               2019-09-30 23:00:00    45.50
Name: pm25, Length: 745920, dtype: float64

In [11]:
df_raw.size / df_raw.index.levels[0].size

12432.0

In [10]:
flist_imputed = glob.glob('ImputedSTHM_trdat/Fitted_fit2/*.csv')
flist_imputed.sort()
flist_imputed

['ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_113E.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_1FD7.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_20CA.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_2E9C.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_3ACF.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_498F.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_4BE7.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_56C3.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_5D7A.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_603A.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_72CA.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_8E2A.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_91B8.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_97D7.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_A838.csv',
 'ImputedSTHM_trdat/Fitted_fit2/ImputedSTHM_trdat_A9BE.csv',
 'ImputedSTHM_trdat/Fitt

In [17]:
se_total = 0
count_total = 0

for fpath in flist_imputed:
    monitor_id = os.path.basename(fpath).split('_', 2)[2][:-4]
    if monitor_id == 'EastArjunNagar_CPCB':
        continue
    df_monitor_impute = pd.read_csv(fpath, index_col=0, parse_dates=True)['fitted']
    df_monitor_impute.sort_index(inplace=True)
    se_total += np.nansum((df_monitor_impute.values - df_raw.loc[monitor_id].values)**2)
    count_total += df_raw.loc[monitor_id].count()

print(np.sqrt(se_total / count_total))

36.70064101311376
